<a href="https://colab.research.google.com/github/Fuenfgeld/2022TeamADataEngineeringBC/blob/37-datavault/DataVault.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataVault

## Introduction

According to Wikipedia DataVault modeling "is a database modeling method that is designed to provide long-term historical storage of data coming in from multiple operational systems". In other words a DataVault is a modelling technique which allows us to import all (structured) raw data into a database without the need to clean or alter the data. The DataVault's main purpose is the store the unprocessed raw data long-term without the need to alter the data once it has been imported.

It achieve this purpose all data that has been imported into a DataVault may **never** be altered. Even schema changes in a source system can be modelled in a DataVault without the need to adjust and update historical data.

**Important:** A DataVaults purpose is the collect data from OLTP systems and save it. The DataVault is **never** used as an OLTP system.

## Benefits

1. Long-term storage of data - As described in the introduction a DataVault model allows for a change resistent data schema. The schema can handle changes in data, changes in data source and changes in the schemas of source systems.
2. Tracking data changes - The DataVault tracks every single data change within a source system even if the source system does not implement a change detection itself. The DataVault not only detects changes but also when these changes occure which make the system well suited if a dataset is supposed to be audible.
3. Fast import of data - Another almost accidental benefit of the DataVault is that data can be imported very quickly. Since a DataVault always has a pre-existing version of a dataset, only the changes need to be synchronized leading to a high import performance.
4. Changes in data schemas do not necessarily required downstream ETL and analysis processes to be adjusted. - Since the DataVault handles alterations of the source schemas very well, downstream ETL processes might not have to be adjusted. In most cases the conversion of the source data schema into the DataVault schema will swallow all schema changes (while keeping all data) and thereby provide a stable interface for downstream ETL and analysis processes.

## Drawbacks

1. Not easy to query - The DataVault modeling creates rather complex schemas which make is hard to query data. A simple data table with an identifier and some data will be projected onto multiple tables in the DataVault context. Thereby reading from a DataVault becomes rather difficult.
2. Large overhead - As the previous point already leads on, a DataVault is rather complex and has a lot of overhead. This is due to the fact, that the DataVault is meant to resistant to changes. Therefore, relationships, data and identifiers are split into separate tables and thereby creating a much higher complexity than exists in any of the source systems.

## Modeling

To explain a model, we will take two simple tables which are connected by a relationship.

```
crops:
+------+----------+-------------------+
| id   | crop     | water_consumption |
+------+----------+-------------------+
| C-1  | tomato   | 10                |
+------+----------+-------------------+
| C-2  | cucumber | 15                |
+------+----------+-------------------+

fields:
+------+-------+----------+
| id   | field | crop_id  |
+------+-------+----------+
| F-5  | small | 2        |
+------+-------+----------+
| F-6  | big   | 1        |
+------+-------+----------+
```

As you can see we have a table which contains crop that we are growing and a table with our fields. Since each field has one crop growing on it the tables have a n to 1 relationship.

### Hubs - Separating business keys

The first thing we do to convert our source model into a DataVault Model is to extract the business keys into their own table. In addition to the business keys we also save the source system as well as the load date and some other information.

To simplify our example, we will only demonstrate a small amount of the fields. If you want to see all the other fields which should be saved on a hub, please refer to the DataVault book.

```
crops hub:

+----------+----------+------------+---------------+
| hash_key | crop_key | load_date  | record_source |
+----------+----------+------------+---------------+
| b519e    | C-1      | 2022-05-01 | ERP           |
+----------+----------+------------+---------------+
| 5f763    | C-2      | 2022-05-01 | ERP           |
+----------+----------+------------+---------------+
```

The hash key is used so the the importing system can easily check if a key has already been imported. Please also not that there is no `id` field. The primary key is the `hash_key` here.

### Satelites - Importing the data

Since we split apart the business keys from the data, we still need to import the data. That is done using the a satelite which contains all columns from the source table as well as a few additional columns: the load date, end date, hash diff.

Satelites will never contain relationship data. Therefore, in case of the `fields` table the `crop_id` column would be dropped here. Additionally, every data column needs to be nullable to allow for later changes in the data schema.

```
crops satelite:
+---------------+----------+-------------------+------------+----------+---------------+-----------+
| crop_hash_key | crop     | water_consumption | load_date  | end_date | record_source | hash_diff |
+---------------+----------+-------------------+------------+----------+---------------+-----------+
| b519e         | tomate   | 10                | 2022-05-01 | NULL     | ERP           | 8a3f0     |
+---------------+----------+-------------------+------------+----------+---------------+-----------+
| 5f763         | cucumber | 15                | 2022-05-01 | NULL     | ERP           | c345a     |
+---------------+----------+-------------------+------------+----------+---------------+-----------+
```

The primary key is a multi column key using the `hash_key` and the `load_date` columns.

### Links - Saving relationships between tables

Finally, we will save the relationship between tables. This is done in another table.

```
crops fields link:
+---------------------+---------------+----------------+------------+---------------+
| field_crop_hash_key | crop_hash_key | field_hash_key | load_date  | record_source |
+---------------------+---------------+----------------+------------+---------------+
| c8fe3               | b519e         | 9f151          | 2022-05-01 | ERP           |
+---------------------+---------------+----------------+------------+---------------+
| eaeef               | 5f763         | 668f0          | 2022-05-01 | ERP           |
+---------------------+---------------+----------------+------------+---------------+
```

The primary key here is the `field_crop_hash_key` column.

## Accommodating changes

Here are some examples of how changes in the source system affect the DataVault models. (An insert is not presented here since it is trivial.)

### Changes in a data set

_Szenario:_ Let's say we recognize that tomatos don't consume 10 entites of water but 12.

_Approach:_ We set the `end_date` of the current row and add a new row will the new values.

_Before data changes:_
```
crops satelite:
+---------------+----------+-------------------+------------+----------+---------------+-----------+
| crop_hash_key | crop     | water_consumption | load_date  | end_date | record_source | hash_diff |
+---------------+----------+-------------------+------------+----------+---------------+-----------+
| b519e         | tomate   | 10                | 2022-05-01 | NULL     | ERP           | 8a3f0     |
+---------------+----------+-------------------+------------+----------+---------------+-----------+
...
+---------------+----------+-------------------+------------+----------+---------------+-----------+
```

_After data changes:_
```
crops satelite:
+---------------+----------+-------------------+------------+------------+---------------+-----------+
| crop_hash_key | crop     | water_consumption | load_date  | end_date   | record_source | hash_diff |
+---------------+----------+-------------------+------------+------------+---------------+-----------+
| b519e         | tomate   | 10                | 2022-05-01 | 2022-05-02 | ERP           | 8a3f0     |
+---------------+----------+-------------------+------------+------------+---------------+-----------+
...
+---------------+----------+-------------------+------------+------------+---------------+-----------+
| b519e         | tomate   | 12                | 2022-05-02 | NULL       | ERP           | 8a3f0     |
+---------------+----------+-------------------+------------+------------+---------------+-----------+
```

As you can see the `hash_key` has not changed since this new data is now associated with the business key. Only the `end_date` on the old row has been set (which is the **only** update operation allowed in the whole DataVault model) to show that the data set is no longer active.

_Benefits:_
- ✅ Tracking data changes
- ✅ Only import differences

### Add columns

_Szenario:_ Let's say we also want to save the crop height.

_Approach:_ We create a new satelite table with the additional data and connect the new table with the old table using a new link table.

_New tables:_
```
crops height satelite:
+-----------------+-------------+------------+----------+---------------+-----------+
| height_hash_key | crop_height | load_date  | end_date | record_source | hash_diff |
+-----------------+-------------+------------+----------+---------------+-----------+
| 74f10           | 30          | 2022-05-03 | NULL     | ERP           | a8c94     |
+-----------------+-------------+------------+----------+---------------+-----------+
| d9570           | 15          | 2022-05-03 | NULL     | ERP           | ef3a9     |
+-----------------+-------------+------------+----------+---------------+-----------+

crops - crops height link:
+---------------------------+---------------+----------------------+------------+---------------+
| crop_crop_height_hash_key | crop_hash_key | crop_height_hash_key | load_date  | record_source |
+---------------------------+---------------+----------------------+------------+---------------+
| 05610                     | b519e         | 74f10                | 2022-05-03 | ERP           |
+---------------------------+---------------+----------------------+------------+---------------+
| 2c61c                     | 5f763         | d9570                | 2022-05-03 | ERP           |
+---------------------------+---------------+----------------------+------------+---------------+
```

Using this approach we just added a new table to accommodate the change in the source schema. Everything else stays the same.

_Benefits:_
- ✅ Tracking schema changes
- ✅ Downstream ETL processes do not need to be adjusted if they don't need the new information

### Remove columns

_Szenario:_ Let's say we don't need the `water_consumption` column anymore.

_Approach:_ Since all columns need to be nullable anyway, we don't need to adjust our model and can just import `NULL` values into the correct columns. But again, we will add a new row, for each data entry in order to not destroy our historical data.

_Benefits:_
- ✅ Tracking schema changes
- ✅ Downstream ETL processes do not need to be adjusted

### Changes relationship dimensions

_Szenario:_ Let's say we want to split the second field to grow both tomatos and cucumbers.

_Approach:_ We just add another row the the link table and from now on the DataVault will contain an n:m relationship.

_Before relationship changes:_
```
crops fields link:
+---------------------+---------------+----------------+------------+---------------+
| field_crop_hash_key | crop_hash_key | field_hash_key | load_date  | record_source |
+---------------------+---------------+----------------+------------+---------------+
...
+---------------------+---------------+----------------+------------+---------------+
| eaeef               | 5f763         | 668f0          | 2022-05-01 | ERP           |
+---------------------+---------------+----------------+------------+---------------+
```

_After data changes:_
```
crops fields link:
+---------------------+---------------+----------------+------------+---------------+
| field_crop_hash_key | crop_hash_key | field_hash_key | load_date  | record_source |
+---------------------+---------------+----------------+------------+---------------+
...
+---------------------+---------------+----------------+------------+---------------+
| eaeef               | 5f763         | 668f0          | 2022-05-01 | ERP           |
+---------------------+---------------+----------------+------------+---------------+
| f2483               | b519e         | 668f0          | 2022-05-01 | ERP           |
+---------------------+---------------+----------------+------------+---------------+
```

_Benefits:_
- ✅ Tracking data changes
- ✅ Only import differences

### Remove relationships

This szenario is a more advanced szenario which is why it is only covered conceptually. Generally speaking there are two ways to cover this szenario:
1. You can add an `end_date` to the link table. This, however, is strongly discuraged since has multiple edge cases in which the DataVault model would fail. One of these is what is a crop is grown on a field then not anymore an then again? If the `field_crop_hash_key` column is generate as a random value. That is possible. However, this leads to bad performance during imports since usually the `hash_key` of any of the tables is actually a hash of the important and not changing fields. If that is the `hash_key` is an actual has, this approach does not work.
2. You will need to add a new hub and satelite with metadata for the relationship and therefore for this link.



# Sources

- Wikipedia: https://en.wikipedia.org/wiki/Data_vault_modeling
- https://www.sciencedirect.com/book/9780128025109/building-a-scalable-data-warehouse-with-data-vault-2-0
- https://danlinstedt.com/wp-content/uploads/2018/06/DVModelingSpecs2-0-1.pdf